In [1]:
#Stock Price Prediction
#Predict stock price of top technology companies(Amazon, Facebook, Apple) by analysing their historic data.
#Predict price for 2017, 2018, 2019 & 2020 without social sentiment
#Predict price for 2017, 2018, 2019 & 2020 with social sentiment .
#Physically validate the stock price derived using option 1& 2 and conclude which is actually close to the current market prices.


In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import collections




from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC


from bs4 import BeautifulSoup
import requests



In [3]:
#Get the data for amazon and create dataframe 
#Add company name as feature to the dataframe

import pandas as pd

data_amzn = pd.read_csv("AMZN.csv")

data_amzn.insert(0,'Stock','AMZN')

data_amzn




,Stock,Date,Open,High,Low,Close,Adj Close,Volume
0,AMZN,2010-12-06,175.520004,178.429993,174.600006,178.050003,178.050003,5654200
1,AMZN,2010-12-07,180.500000,181.470001,176.570007,176.770004,176.770004,5077100
2,AMZN,2010-12-08,177.490005,178.160004,175.199997,176.289993,176.289993,3726900
3,AMZN,2010-12-09,177.770004,178.110001,173.800003,174.850006,174.850006,4555500
4,AMZN,2010-12-10,174.880005,175.949997,173.360001,175.619995,175.619995,3565500
...,...,...,...,...,...,...,...,...
2513,AMZN,2020-11-30,3208.479980,3228.389893,3125.550049,3168.040039,3168.040039,4063900
2514,AMZN,2020-12-01,3188.500000,3248.949951,3157.179932,3220.080078,3220.080078,4544400
2515,AMZN,2020-12-02,3221.649902,3232.000000,3173.260010,3203.530029,3203.530029,3129300
2516,AMZN,2020-12-03,3205.459961,3228.639893,3181.310059,3186.729980,3186.729980,2892000


In [4]:
#Get the data for google and create dataframe 
#Add company name as feature to the dataframe


data_goog = pd.read_csv("GOOG.csv")

data_goog.insert(0,'Stock','GOOG')

data_goog


,Stock,Date,Open,High,Low,Close,Adj Close,Volume
0,GOOG,2010-01-04,312.304413,313.579620,310.954468,312.204773,312.204773,3927000
1,GOOG,2010-01-05,312.418976,312.747742,309.609497,310.829926,310.829926,6031900
2,GOOG,2010-01-06,311.761444,311.761444,302.047852,302.994293,302.994293,7987100
3,GOOG,2010-01-07,303.562164,303.861053,295.218445,295.940735,295.940735,12876600
4,GOOG,2010-01-08,294.894653,300.498657,293.455048,299.885956,299.885956,9483900
...,...,...,...,...,...,...,...,...
2747,GOOG,2020-12-01,1774.369995,1824.829956,1769.369995,1798.099976,1798.099976,1739000
2748,GOOG,2020-12-02,1798.099976,1835.650024,1789.473022,1827.949951,1827.949951,1222000
2749,GOOG,2020-12-03,1824.010010,1847.199951,1822.650024,1826.770020,1826.770020,1227300
2750,GOOG,2020-12-04,1824.520020,1833.160034,1816.989990,1827.989990,1827.989990,1378200


In [5]:
#Get the data for facebook and create dataframe 
#Add company name as feature to the dataframe


data_fb = pd.read_csv("FB.csv")

data_fb.insert(0,'Stock','FB')

data_fb

,Stock,Date,Open,High,Low,Close,Adj Close,Volume
0,FB,2012-05-21,36.529999,36.660000,33.000000,34.029999,34.029999,168192700
1,FB,2012-05-22,32.610001,33.590000,30.940001,31.000000,31.000000,101786600
2,FB,2012-05-23,31.370001,32.500000,31.360001,32.000000,32.000000,73600000
3,FB,2012-05-24,32.950001,33.209999,31.770000,33.029999,33.029999,50237200
4,FB,2012-05-25,32.900002,32.950001,31.110001,31.910000,31.910000,37149800
...,...,...,...,...,...,...,...,...
2147,FB,2020-12-01,279.160004,289.299988,278.959991,286.549988,286.549988,20777900
2148,FB,2020-12-02,285.359985,291.779999,280.829987,287.519989,287.519989,17361600
2149,FB,2020-12-03,286.250000,286.649994,281.070007,281.850006,281.850006,12921700
2150,FB,2020-12-04,280.299988,283.459991,279.299988,279.700012,279.700012,10880300


In [6]:
#Get the data for apple and create dataframe 
#Add company name as feature to the dataframe


data_aapl = pd.read_csv("AAPL.csv")

data_aapl.insert(0,'Stock','APPL')

data_aapl

,Stock,Date,Open,High,Low,Close,Adj Close,Volume
0,APPL,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.539882,493729600
1,APPL,2010-01-05,7.664286,7.699643,7.616071,7.656428,6.551187,601904800
2,APPL,2010-01-06,7.656428,7.686786,7.526786,7.534643,6.446983,552160000
3,APPL,2010-01-07,7.562500,7.571429,7.466072,7.520714,6.435065,477131200
4,APPL,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.477847,447610800
...,...,...,...,...,...,...,...,...
2747,APPL,2020-12-01,121.010002,123.470001,120.010002,122.720001,122.720001,128166800
2748,APPL,2020-12-02,122.019997,123.370003,120.889999,123.080002,123.080002,89004200
2749,APPL,2020-12-03,123.519997,123.779999,122.209999,122.940002,122.940002,78967600
2750,APPL,2020-12-04,122.599998,122.860001,121.519997,122.250000,122.250000,78260400


In [7]:
#Merge the data_amzn, data_goog, data_fb , data_aapl into single dataframe

data = pd.concat([data_amzn, data_goog, data_fb , data_aapl])

data

,Stock,Date,Open,High,Low,Close,Adj Close,Volume
0,AMZN,2010-12-06,175.520004,178.429993,174.600006,178.050003,178.050003,5654200
1,AMZN,2010-12-07,180.500000,181.470001,176.570007,176.770004,176.770004,5077100
2,AMZN,2010-12-08,177.490005,178.160004,175.199997,176.289993,176.289993,3726900
3,AMZN,2010-12-09,177.770004,178.110001,173.800003,174.850006,174.850006,4555500
4,AMZN,2010-12-10,174.880005,175.949997,173.360001,175.619995,175.619995,3565500
...,...,...,...,...,...,...,...,...
2747,APPL,2020-12-01,121.010002,123.470001,120.010002,122.720001,122.720001,128166800
2748,APPL,2020-12-02,122.019997,123.370003,120.889999,123.080002,123.080002,89004200
2749,APPL,2020-12-03,123.519997,123.779999,122.209999,122.940002,122.940002,78967600
2750,APPL,2020-12-04,122.599998,122.860001,121.519997,122.250000,122.250000,78260400


In [20]:
# Data Cleaning : Detecting Missing Values or any other Data Quality Issue

#Check datatypes.
print(data.dtypes)
#print(data.info)


# To find if there is any missing values in complete data True: Yes there is & False : No there is no missing value?
print("Any missing value True/False: ", data.isnull().values.any())

#Sum of NULL or Missing values in dataframe across all col.
data.isnull().sum()


Stock         object
Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object
Any missing value True/False:  False


Stock        0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64